In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os

def build_generator(img_shape):
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, input_dim=100, activation='relu'))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(np.prod(img_shape), activation='tanh'))
    model.add(layers.Reshape(img_shape))
    return model

def build_discriminator(img_shape):
    model = tf.keras.Sequential()
    model.add(layers.Flatten(input_shape=img_shape))
    model.add(layers.Dense(512,  activation='relu'))
    model.add(layers.Dense(256,  activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

def build_gan(generator, discriminator):
    discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Nurodo, kad discrimanator modelis nesimokins. Tai padeda isvengti perteklinio mokymo
    discriminator.trainable = False
    gan_input = tf.keras.Input(shape=(100,))
    img = generator(gan_input)
    gan_output = discriminator(img)
    gan = tf.keras.Model(gan_input, gan_output)
    gan.compile(loss='binary_crossentropy', optimizer='adam')
    return gan

# nurodom nuotrauku ismatavimus
img_shape = (64, 64, 3)
# nurodom kelia iki nuotrauku
image_dir = '../51_paskaita/autoplius/automobiliai'

def load_images(image_dir, img_shape):
    images_paths = [os.path.join(image_dir, image)  for image in os.listdir(image_dir) if f.lower().endswith('jpg')]
    images = []
    for image_path in images_paths:
        try:
            img = tf.keras.preprocessing.image.load_img(image_path, target_size=img_shape[:2])
            img = tf.keras.preprocessing.image.img_to_array(img)
            images.append(img)
        except Exception as e:
            print(f'Nuotrauka praleidziama, del klaidos {e}')
    images = np.array(images)
    images = (images.astype('float32') -127.5) / 127.5 # normalizuoja duomenis nuo -1 iki 1
    return images

load_images(image_dir, img_shape)
